In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_11월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(1084190, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
1084190


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,0,"[과천, 서울대, 공원, 딜쿠샤, 철도, 노조, 파업, 2022, 11, 30, 수...","[('과천', 'Noun'), ('서울대', 'Noun'), ('공원', 'Noun..."
1,서울은 지하철 타기가 전쟁통을 방불케 한다,0,"[서울, 은, 지하철, 타기가, 전쟁, 통, 을, 방불, 케, 한다]","[('서울', 'Noun'), ('은', 'Josa'), ('지하철', 'Noun'..."
2,그래도 질서 있게 줄 섰다가 다음 열차를 타라는 안내 방송이 나오면 타기를 중단하고...,0,"[그래도, 질서, 있게, 줄, 섰다가, 다음, 열차, 를, 타라, 는, 안내, 방송...","[('그래도', 'Adverb'), ('질서', 'Noun'), ('있다', 'Ad..."
3,서울대공원 안의 호수 메타세쿼이아 길 딜쿠샤는 기쁜 마음 이라는 뜻의 페르시아어로 ...,0,"[서울대, 공원, 안, 의, 호수, 메타세쿼이아, 길, 딜쿠샤, 는, 기쁜, 마음,...","[('서울대', 'Noun'), ('공원', 'Noun'), ('안', 'Noun'..."
4,앨버트 테일러는 3 1운동을 뉴욕타임즈에 보도함으로써 우리 나라의 항일운동을 세계에...,0,"[앨버트, 테일러, 는, 3, 1, 운동, 을, 뉴욕타임즈, 에, 보도, 함, 으로...","[('앨버트', 'Noun'), ('테일러', 'Noun'), ('는', 'Josa..."
...,...,...,...,...
1084185,갑신정변이 실패하고 홍영식이 비참하게 죽은 뒤 민영익의 도움을 받은 알렌은 흉가가 ...,22958,"[갑신정변, 이, 실패하고, 홍영식, 이, 비참하게, 죽은, 뒤, 민영익, 의, 도...","[('갑신정변', 'Noun'), ('이', 'Josa'), ('실패하다', 'Ad..."
1084186,제중원의 뜰에는 나무 한그루가 서 있었다,22958,"[제중원, 의, 뜰, 에는, 나무, 한그루, 가, 서, 있었다]","[('제중원', 'Noun'), ('의', 'Josa'), ('뜰', 'Noun')..."
1084187,박지원의 집앞에 있었던 나무 홍영식의 집앞에 있었던 나무 그날 길 잃은 아이들처럼 ...,22958,"[박지원, 의, 집앞, 에, 있었던, 나무, 홍영식, 의, 집앞, 에, 있었던, 나...","[('박지원', 'Noun'), ('의', 'Josa'), ('집앞', 'Noun'..."
1084188,그녀와 내가 헤어진 지금 이 모든 일이 과연 우연일 뿐이라고 생각하는가 21쪽 쉽게...,22958,"[그녀, 와, 내, 가, 헤어진, 지금, 이, 모든, 일이, 과연, 우연, 일, 뿐...","[('그녀', 'Noun'), ('와', 'Josa'), ('내', 'Noun'),..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(165622, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '때', '등', '우리', '말', '길', '나', '더', '생각', '내', '문화', '시간', '층', '위', '사진', '나무', '의', '전', '여행', '제', '안', '중', '때문', '하나', '일', '집', '카페', '일반', '날', '마을', '및', '위해', '저', '로', '모습', '공원', '다른', '작품', '아이', '공간', '시작', '한국', '개', '또', '앞']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 1084190/1084190 [00:08<00:00, 133846.95it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

48
43


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 1084190/1084190 [13:01<00:00, 1386.82it/s]


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/11월/단어동시출현_딕셔너리_11월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 222796235/222796235 [23:16<00:00, 159551.02it/s] 


(222796235, 3)


,term1,term2,freq
0,30,넘었고,1
1,30,들으며,14
2,11,30,1996
3,30,사는,63
4,30,철도,48


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/11월/단어동시출현_df_11월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 222796235/222796235 [41:31<00:00, 89428.17it/s]


,term1,term2,freq
0,공원,집,622
1,사람,이,46956
2,모습,이,15610
3,의,이,219047
4,문화,이,19280
...,...,...,...
1220,공원,아이,693
1221,공원,다른,354
1222,일반,카페,362
1223,아이,일반,198


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/11월/단어동시출현_df_상위50_11월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1514.00it/s]


In [16]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,때,등,우리,말,길,나,더,생각,내,문화,시간,층,위,사진,나무,의,전,여행,제,안,중,때문,하나,일,집,카페,일반,날,마을,및,위해,저,로,모습,공원,다른,작품,아이,공간,시작,한국,개,또,앞
것,0.0,31032.0,30298.0,117047.0,10924.0,18416.0,15503.0,9042.0,14488.0,15648.0,4872.0,16419.0,11375.0,13560.0,13202.0,6128.0,7065.0,1956.0,8908.0,4584.0,5661.0,85284.0,7608.0,3667.0,5998.0,8513.0,7961.0,9511.0,6840.0,9677.0,4246.0,1860.0,1333.0,4398.0,2292.0,2409.0,6273.0,6096.0,34354.0,4463.0,1780.0,7726.0,3855.0,3882.0,3420.0,4857.0,3640.0,3578.0,5413.0,5154.0
수,31032.0,0.0,17386.0,74256.0,13740.0,11332.0,9328.0,11193.0,8831.0,8177.0,5914.0,11261.0,8550.0,7826.0,8353.0,7297.0,7590.0,3313.0,6567.0,4851.0,5702.0,63890.0,6620.0,3947.0,5422.0,6046.0,5894.0,6512.0,4382.0,6933.0,3521.0,2842.0,1346.0,3357.0,3119.0,3675.0,4246.0,3729.0,30027.0,4702.0,3545.0,5089.0,4111.0,4230.0,6123.0,3294.0,3171.0,3837.0,3821.0,3895.0
그,30298.0,17386.0,0.0,64909.0,9824.0,13231.0,10817.0,6085.0,8517.0,10112.0,3818.0,9930.0,5616.0,6492.0,7930.0,3840.0,3878.0,1318.0,6075.0,2606.0,3819.0,60579.0,5208.0,1874.0,4066.0,5593.0,7065.0,5571.0,4248.0,5982.0,3272.0,1165.0,779.0,3266.0,1667.0,1641.0,4155.0,3099.0,24103.0,3628.0,1112.0,4347.0,3014.0,2155.0,2113.0,3200.0,2220.0,2477.0,3653.0,3547.0
이,117047.0,74256.0,64909.0,0.0,51320.0,46956.0,39363.0,31580.0,29286.0,30843.0,19194.0,38183.0,26584.0,28645.0,29067.0,19280.0,23328.0,8972.0,20950.0,18238.0,19467.0,219047.0,23137.0,11735.0,18357.0,23614.0,21656.0,19839.0,15675.0,20943.0,14239.0,9347.0,3450.0,17376.0,10858.0,8584.0,14613.0,17080.0,98884.0,15610.0,9352.0,15998.0,12214.0,12602.0,13588.0,14297.0,11244.0,13017.0,13638.0,14311.0
곳,10924.0,13740.0,9824.0,51320.0,0.0,5628.0,5178.0,4653.0,3230.0,2690.0,5164.0,5103.0,4054.0,3540.0,3891.0,4158.0,4032.0,1656.0,2867.0,3949.0,3391.0,29464.0,3781.0,4967.0,3230.0,3592.0,4249.0,2212.0,2420.0,2453.0,2440.0,2908.0,348.0,2083.0,3475.0,1208.0,2134.0,3229.0,16834.0,2554.0,3411.0,2949.0,1100.0,2907.0,3101.0,1881.0,1540.0,2087.0,1994.0,2371.0
사람,18416.0,11332.0,13231.0,46956.0,5628.0,0.0,6411.0,3067.0,5389.0,6779.0,2630.0,7320.0,4175.0,5523.0,5195.0,2528.0,2821.0,819.0,3090.0,2740.0,2373.0,31430.0,2999.0,1415.0,2041.0,3983.0,3332.0,3310.0,2505.0,4238.0,2222.0,999.0,444.0,2231.0,1483.0,711.0,2777.0,2494.0,13304.0,2172.0,851.0,4131.0,1204.0,1175.0,1424.0,1896.0,1436.0,1436.0,2360.0,2333.0
때,15503.0,9328.0,10817.0,39363.0,5178.0,6411.0,0.0,3216.0,4867.0,4887.0,2337.0,6520.0,3726.0,4477.0,5140.0,1888.0,2851.0,755.0,2639.0,2338.0,2259.0,26586.0,3255.0,1777.0,2775.0,3575.0,2966.0,2540.0,2127.0,3626.0,2252.0,892.0,421.0,2169.0,982.0,714.0,2113.0,2462.0,13282.0,1791.0,807.0,2380.0,1045.0,1629.0,1057.0,2311.0,1320.0,1500.0,2028.0,2034.0
등,9042.0,11193.0,6085.0,31580.0,4653.0,3067.0,3216.0,0.0,2024.0,2455.0,1781.0,4370.0,1954.0,1596.0,2768.0,4809.0,2101.0,1336.0,3655.0,1552.0,2886.0,31252.0,3969.0,1305.0,3196.0,2018.0,3136.0,1487.0,1356.0,2531.0,1425.0,1160.0,776.0,1051.0,1726.0,4409.0,2293.0,1113.0,14092.0,1642.0,1728.0,1431.0,2104.0,1254.0,2353.0,1596.0,2515.0,2767.0,1080.0,1160.0
우리,14488.0,8831.0,8517.0,29286.0,3230.0,5389.0,4867.0,2024.0,0.0,4236.0,1957.0,4429.0,3347.0,3918.0,3436.0,2764.0,2663.0,794.0,2473.0,1785.0,1807.0,25935.0,2415.0,1251.0,1602.0,2944.0,2159.0,2194.0,2102.0,3019.0,2480.0,882.0,387.0,1946.0,919.0,643.0,2174.0,1515.0,9996.0,1432.0,573.0,1977.0,951.0,1821.0,1030.0,1548.0,1324.0,1086.0,1784.0,1908.0
말,15648.0,8177.0,10112.0,30843.0,2690.0,6779.0,4887.0,2455.0,4236.0,0.0,1418.0,5806.0,2747.0,3466.0,4804.0,1614.0,1557.0,393.0,2495.0,981.0,1434.0,24330.0,2275.0,741.0,1971.0,2898.0,2260.0,2235.0,1867.0,3007.0,1351.0,386.0,337.0,1419.0,576.0,566.0,1686.0,1708.0,9514.0,1050.0,427.0,2066.0,754.0,1135.0,601.0,1306.0,944.0,1144.0,1677.0,1599.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/11월/공출현빈도_df_11월.csv", encoding='UTF-8')